In [1]:
import numpy as np
import pandas as pd
import datetime
import math

In [2]:
dam = pd.read_csv('dam.csv')
dam

FileNotFoundError: [Errno 2] File dam.csv does not exist: 'dam.csv'

In [ ]:
dam['date']=pd.to_datetime(dam['tradedate'],format="%Y/%m/%d")#"%d/%m/%Y")
dam=dam.set_index(pd.DatetimeIndex(dam['date']))
dam['year']=dam.index.year
dam['month']=dam.index.month
dam['day']=dam.index.day
dam['day_of_week']=dam.index.dayofweek
dam['month_no']=dam['year']+dam['month']/100

dam['hour_ref'] = dam.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)
dam=dam.drop(['hourlyid','tradedate','date'],axis=1)
dam

In [ ]:
dam.loc[dam['hour']<12].groupby(['day','month','year']).max()['price']


In [ ]:
dam['value']=dam['price']*dam['volume']
daily=dam.groupby(['year','month','day']).sum().drop(['hour','day_of_week','month_no','market'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=dam[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=dam[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=dam[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=dam[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=dam[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=dam[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=dam.loc[dam['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=dam.loc[dam['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=dam.loc[(dam['hour']>=12) & (dam['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()

daily

In [ ]:
dam=dam.reset_index()#.set_index(['year','month','day'])
dam

In [ ]:

#dam['price_min']=daily['price_min']
#dam['price_max']=daily['price_max']
#dam['price_mean']=daily['price_mean']
#dam['price_standard']=daily['price_standard']

#dam['price_morning_max']=daily['price_morning_max']
#dam['price_evening_max']=daily['price_evening_max']


dam['cat']='off'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==6),'cat']='standard'
dam.loc[(dam['day_of_week']<5) & (dam['hour']>=7) & (dam['hour']<=9),'cat']='morning'

dam.loc[(dam['day_of_week']<5) & (dam['hour']>=10) & (dam['hour']<=16),'cat']='standard'

dam.loc[(dam['day_of_week']<5) & (dam['hour']==17) & (dam['month']>=6) & (dam['month']<=8),'cat']='evening'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==17) & ((dam['month']<=5) | (dam['month']>=9)),'cat']='standard'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==18) & (dam['month']<=3),'cat']='standard'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==18) & (dam['month']>=4),'cat']='evening'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==19),'cat']='evening'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==20) & (dam['month']<=3),'cat']='evening'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==20) & (dam['month']>=4),'cat']='standard'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==21),'cat']='standard'

dam.loc[(dam['day_of_week']==5) & (dam['hour']>=7) & (dam['hour']<=11),'cat']='standard'
dam.loc[(dam['day_of_week']==5) & (dam['hour']==17) & (dam['month']>=6) & (dam['month']<=8),'cat']='standard'
dam.loc[(dam['day_of_week']==5) & (dam['hour']==18) & (dam['month']>=4),'cat']='standard'
dam.loc[(dam['day_of_week']==5) & (dam['hour']==19),'cat']='standard'
dam.loc[(dam['day_of_week']==5) & (dam['hour']==20) & (dam['month']<=3),'cat']='standard'






#dam['cat']='standard'
#dam.loc[(dam['hour']<11) & (dam['price_morning_max'] > dam['price_standard'] + (dam['price_standard'] - dam['price_min'])) & (dam['price'] > dam['price_morning_max']-(dam['price_mean'] * 0.2)),'cat']='morning'
#dam.loc[(dam['hour']>=11) & (dam['price_evening_max'] > dam['price_standard'] + (dam['price_standard'] - dam['price_min'])) & (dam['price'] > dam['price_evening_max']-(dam['price_mean'] * 0.2)),'cat']='evening'

#dam.loc[dam['price'] < dam['price_min']+(dam['price_mean'] * 0.2),'cat']='off'

#dam.loc[dam['price_max'] - dam['price_min']<30,'cat']='off'
dam

In [ ]:
daily['off_price_mean']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=dam.loc[dam['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=dam.loc[dam['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=dam.loc[dam['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=dam.loc[dam['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=dam.loc[dam['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=dam.loc[dam['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=dam.loc[dam['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=dam.loc[dam['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

#daily=daily.drop(['price_min','price_max','price_morning_max','price_evening_max'], axis=1)

daily

In [ ]:
daily.loc[2016,2,8]

In [ ]:
daily.to_csv('dam_daily.csv')

In [ ]:
dam=dam.drop(['hour_ref','month_no'],1)

In [ ]:
dam.to_csv('dam_out.csv')

In [ ]:
dam.info()

In [ ]:
dam_flat=dam.reset_index().set_index(['date'])
dam_slim=dam_flat.loc[(dam_flat['year']<2021)][['hour','price','volume','cat']]
dam_slim

In [ ]:
dam_slim.to_csv('dam_slim.csv')